# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [1]:
# Run this cell first!
%matplotlib inline
from helpers import Map, load_map_10, load_map_40, show_map
import math
import numpy as np

%load_ext autoreload
%autoreload 2

### Map Basics

In [2]:
map_10 = load_map_10()
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections
# nodes = intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

In [4]:
list(map_10.intersections.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [5]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[5, 6, 7]

In [6]:
# This shows the full connectivity of the map
map_10.roads

[[5, 6, 7],
 [2, 3, 4],
 [1, 3, 4],
 [1, 2, 4, 5],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [7]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [8]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

## The Algorithm
### Writing your algorithm
The algorithm written will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`. However you must complete several methods before it will work.

```bash
> PathPlanner(map_40, 5, 34).path
[5, 16, 37, 12, 34]
```

In [9]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search()
            return self.path
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            current = self.get_current_node()

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        print("No Path Found")
        self.path = None
        return False

Create the following methods:

In [10]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # TODO: return a data structure suitable to hold the set of nodes already evaluated
    return set()

In [11]:
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        return set([self.start])
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

In [12]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    return dict()

In [13]:
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO:  a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    keywords = list(self.map.intersections.keys())
    values = list(float("inf")if i != self.start else 0.0 for i in keywords)
    return dict(zip(keywords,values))

In [14]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    keywords = list(self.map.intersections.keys())
    values = list(float("inf")if i != self.start else self.heuristic_cost_estimate(self.start) for i in keywords)
    return dict(zip(keywords,values))


In [15]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    # TODO: Set map to new value. 
    self.map = M

In [16]:
def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    self.goal = None
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self.start = start

In [17]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    self.goal = goal

In [18]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    openSet_dict=dict({node:self.fScore[node] for node in self.openSet})
    
    return min(openSet_dict, key=openSet_dict.get)

In [19]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    return self.map.roads[node]

In [20]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    return self.gScore[node]

In [21]:
def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the g Score of the current node 
    # plus distance from the current node to it's neighbors
    return self.get_gScore(current) + self.distance(current, neighbor)

In [22]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the open set is empty. False otherwise.
    if len(self.openSet)!=0:
        return False
    return True

In [23]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    coordinate_1 = np.array(self.map.intersections[node_1])
    coordinate_2 = np.array(self.map.intersections[node_2])
    dist = np.linalg.norm(coordinate_2-coordinate_1)
    return dist

In [24]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    return self.distance(node, self.goal)

In [25]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    G = self.get_gScore(node)
    H = self.heuristic_cost_estimate(node)
    return G + H

In [26]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_tenative_gScore( current, neighbor)
    self.fScore[neighbor] = self.calculate_fscore( neighbor)

In [27]:
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

### Display values for testing purposes

In [88]:
import pandas as pd
from IPython.display import display, HTML
planner = PathPlanner(map_40, 8, 24)
print("\nmap: {}".format(planner.map))
print("\nstart: {}".format(planner.start))
print("\ngoal: {}".format(planner.goal ))
print("\nclosedSet: {}".format(planner.closedSet))
print("\nopenSet: {}".format(planner.openSet ))
print("\ncameFrom: {}".format(planner.cameFrom))
print("\n ***********g score **************" )
display(HTML(pd.DataFrame(list(planner.gScore.items()),columns=['node', 'gValue']).to_html(index=False)))
print("\n ***********f score **************" )
display(HTML(pd.DataFrame(list(planner.fScore.items()),columns=['node', 'gValue']).to_html(index=False)))

print"\nPath plan: ", planner.path 


map: <helpers.Map instance at 0x7f3ead349128>

start: 8

goal: 24

closedSet: set([33, 34, 37, 8, 10, 12, 14, 16, 17, 30])

openSet: set([0, 1, 5, 13, 15, 18, 22, 24, 25, 26, 27, 28, 29, 31])

cameFrom: {0: 34, 1: 17, 5: 14, 10: 17, 12: 37, 13: 10, 14: 8, 15: 17, 16: 14, 17: 12, 18: 17, 22: 37, 24: 10, 25: 17, 26: 34, 27: 10, 28: 12, 29: 37, 30: 8, 31: 12, 33: 8, 34: 12, 37: 16}

 ***********g score **************


node,gValue
0,1.108172
1,1.189989
2,inf
3,inf
4,inf
5,0.361767
6,inf
7,inf
8,0.000000
9,inf



 ***********f score **************


node,gValue
0,1.504752
1,1.601830
2,inf
3,inf
4,inf
5,1.393937
6,inf
7,inf
8,0.794601
9,inf



Path plan:  [8, 14, 16, 37, 12, 17, 10, 24]


### An example solution 

In [47]:
planner = PathPlanner(map_40, 5, 5)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

something is off, your code produced the following:
[5]


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [59]:
from test import test

test(PathPlanner)
show_map(planner.map, start=planner.start, goal=planner.goal, path=planner.path)

All tests pass! Congratulations!


## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **: A-Star is a way of getting a solution path to go from a point A to a point B in a network of paths, attempting to get the optimal path which means the path that represents the lowest cost. For instance, you could think of it as a way of getting the fewest connections to get from a place to another in a subway network. A-Star selects the best path to take according to the combination of the distance to the next subway station and an estimation of how close it's to the final station(i.e. the destination station).    

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **: A-star differs from a uniform cost search by implementing a heuristic to its process of selecting the next  node to visit

- What is a heuristic?

** ANSWER **: A heuristic is a technique that attempts to solve a problem quickly or in an approximated way. A heuristic is a problem dependent solution. [Heuristic](https://en.wikipedia.org/wiki/Heuristic_(computer_science%29)

- What is a consistent heuristic?

** ANSWER **: A consistent heuristic "h()" is a heuristic that must meet the following criteria: 
* h(Node) - h (descendant Node) <= cost(Node, descendant Node)

The difference between the heuristic value of the node and any of its descendant Nodes must be less or equal to the real cost from that node and any of its descendant nodes. [consistent heuristic](https://en.wikipedia.org/wiki/Consistent_heuristic)

- What is a admissible heuristic? 

** ANSWER **: An admissible heuristic "h()" is a heuristic that must meet the following criteria: 
* h(Node) <= h*(Node)

The heuristic of any node in the graph must be less or equal than the real optimal cost "h*()"from the Node to the final goal. [admissible heuristic](https://en.wikipedia.org/wiki/Admissible_heuristic)

- ___ admissible heuristic are consistent.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: Some

- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: All